In [311]:
import tweepy, pandas as pd, numpy as np, json, re, seaborn as sns, datetime, time, requests
from textblob import TextBlob
from datetime import timedelta
#from TwitterAPI import TwitterAPI

Setting up Twitter Credentials

In [2]:

consumer_key = 'sW0oAfGFGSWUHbOGv5JhvpCiW'
consumer_secret = 'V6lRbVNA7WKhOu71Io5QyHAr0x0GazuHDNDkykaAeuJjxbNSea'
access_token = '250349566-Q5lBnCb2TQwUGrroNVQgJ2HHkkZPxgyBJQWZYLWM'
access_token_secret = 'NjoCJw62fcxh2t8JFQmA6jxuwLLE6904o6ETOgq02MlzY'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

Creating a function that returns how many minutes between two time stamp points

In [ ]:
def totalminutes(t1, t2):
    diff = t2-t1
    try:
        mins = diff.hours*60+diff.minutes+diff.seconds/60
    except : 
        pass
    
    try:
        mins = diff.minutes+diff.seconds/60
    except : 
        mins = diff.seconds/60
    return round(mins,2)

Getting the twitter handles of US members of congress

In [312]:
legisl = requests.get("https://theunitedstates.io/congress-legislators/legislators-social-media.json")
legisl_dict = json.loads(legisl.text)
users = list()
for row in legisl_dict:
    try:
        users.append(row['social']['twitter'])
    except: 
        time.sleep(10**-6) #this code is in case they don't have a twitter. I maybe could just leave it blank but do need to have an except

In [406]:
users = users[0:3]

This is the full code to retrieve the data. The cells below test parts of the code a piece at a time. This puts it all together

In [ ]:
all_users = dict()
for user in users:
    now = datetime.datetime.now()
    
    #this first block of code reads in the recent tweets from the user
    user_tweets = list(api.user_timeline(user, count=200, include_rts=False, exclude_replies=True, tweet_mode="extended"))
    time.sleep(1)
    length = len(user_tweets)-1
    lasttweet = user_tweets[length]
    lastdate = lasttweet.created_at 
    lastid = lasttweet.id
    tweets_list = list()
    
    #this block reads in more if they have more than 200 in the last week
    while lastdate> (now - timedelta(days=7)): 
        newtweets = api.user_timeline(user, count=200, include_rts=False, exclude_replies=True, max_id=lastid)
        length = len(newtweets)-1
        lastid=newtweets[length].id
        tweet_list = tweet_list+newtweets
        time.sleep(1)    
    user_tweets=user_tweets+tweet_list
    
    #this block deletes any extra tweets that aren't in the last 7 days
    todelete = list()
    for x in range(0, len(user_tweets)):
            if user_tweets[x].created_at < now-timedelta(days=7):
                todelete.append(x)
    user_tweets = user_tweets[0:min(todelete)]

    #this section of the code takes each tweet and tries to find comments
    touser = "to@"+user
    all_usertweets = dict()
    for tweet in range(0,len(user_tweets)):
        tomatch = user_tweets[tweet].id
        totweets = api.search(q= touser, count=100, 
                              tweet_mode="extended", result_type="recent")
        comments = list()
        for x in range(0,len(totweets)):
            if totweets[x].in_reply_to_status_id==tomatch:
                comments.append(totweets[x].full_text)
        if totweets[len(totweets)-1].id>tomatch:
            while len(comments)<100 and totweets[0].id>tomatch:
                time1 = datetime.datetime.now()
                reference = totweets[len(totweets)-1].id
                totweets = api.search(q= touser, count=100, 
                                      tweet_mode="extended", max_id=reference, result_type="recent")
                for x in range(0,len(totweets)):
                    if totweets[x].in_reply_to_status_id==tomatch:
                        comments.append(totweets[x].full_text)  
                time2 = datetime.datetime.now()
                delay = (time2-time1).seconds+(10**-6)*(time2-time1).microseconds
                time.sleep(5-delay) #set this to make sure we don't go over twitter's limit  
                    
    #this section puts the text and other meta-data with the comments and puts them together in a dictionary.
        thistweetframe = dict()
        thistweetframe['text'] = user_tweets[tweet].full_text
        thistweetframe['Retweets'] = user_tweets[tweet].retweet_count
        thistweetframe['Likes'] = user_tweets[tweet].favorite_count
        thistweetframe['Length'] = len(user_tweets[tweet].full_text)
        thistweetframe['Date'] = user_tweets[tweet].created_at
        thistweetframe['comments'] = comments #later we will add other elements of the tweet to the dictionary
        all_usertweets[tweet] = thistweetframe
    
    all_users[user] = all_usertweets
    time_end = datetime.datetime.now()
    print("{} took {} minutes".format(user,totalminutes(now, time_end)))
